In [33]:
import nltk

In [45]:
with open ('artigos.txt', 'r', encoding = "utf8") as f:
    artigos = f.read()
print(artigos[:50])




imagem 

Temos a seguinte classe que representa


### Precisamos refinar nossa base de dados para isso vamos separar ela por tokens

In [46]:
# Método isapha() que verifica sé for todos os caracteres alfabéticos devolve True caso contrário False
def separa_palavras(lista_tokens):
    lista_palavras = []
    for token in lista_tokens:
        if token.isalpha():
            lista_palavras.append(token)
    return lista_palavras

# o método .tokenize.word_tokenize() separa o texto todo em tokens
lista_tokens = nltk.tokenize.word_tokenize(artigos)
lista_palavras = separa_palavras(lista_tokens)
print(f'O número de palavras é {len(lista_palavras)}')

O número de palavras é 403031


### Após a separação do texto em tokens é necessário fazer a normalização desses tokens, ou seja tranformar todas as letras maiúsculas em minúsculas

In [47]:
def normalizacao(lista_palavras):
    lista_normalizada = []
    for palavra in lista_palavras:
        lista_normalizada.append(palavra.lower())    
    return lista_normalizada

lista_normalizada = normalizacao(lista_palavras)
print(f'O número de palavras é {len(lista_normalizada)}')

O número de palavras é 403031


**Podemos observar que a quantidade de palavras ainda são as mesmas, mas agora precisamos retirar todas as palavras repetidas do nosso texto de teste com isso vamos utilizar a idéia de conjuntos com o método .set()**

In [48]:
print(f'O número de palavras é {len(lista_palavras)}')

O número de palavras é 403031


## Algoritmo do corretor

In [52]:
# A função_lestras(). insere letras até encontrar a palavra correta.
def insere_letras(fatias):
    novas_palavras = []
    letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D)
    return novas_palavras

# A função deletando_caracteres(). remove letras que são digitas a mais que o necessário.
def deletando_caracteres(fatias):
    novas_palavras = []
    for E, D in fatias:
        novas_palavras.append(E + D[1:])
    return novas_palavras

# A função troca_letra(). ele vai trocar letras por exemplo (lígica) em (lógica).
def troca_letra(fatias):
    novas_palavras = []
    letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D[1:])
    return novas_palavras

# A função inverte_letra(). ele vai inverter letras por exemplo (lgóica) em (lógica)
def inverte_letra(fatias):
    novas_palavras = []
    for E, D in fatias:
        if len(D) > 1:
            novas_palavras.append(E + D[1] + D[0] + D[2:])
    return novas_palavras

# A função gerador_turbinado(). Faz duas operaçoes que será capaz de tranformar (lóiigica) em em (lógica).
def gerador_turbinado(palavras_geradas):
    novas_palavras = []
    for palavra in palavras_geradas:
        novas_palavras += gerador_palavras(palavra)
    return novas_palavras

def gerador_palavras(palavra):
    fatias = []
    for i in range(len(palavra)):
        fatias.append((palavra[:i], palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    palavras_geradas += deletando_caracteres(fatias)
    palavras_geradas += troca_letra(fatias)
    palavras_geradas += inverte_letra(fatias)
    return palavras_geradas

def probabilidade(palavra_gerada):
    return frequencia[palavra_gerada]/total_palavras

# A função max vai calcular o máximo de probabilidade da palavra estar correta
def corretor(palavra):
    palavras_geradas = gerador_palavras(palavra)
    palavra_correta = max(palavras_geradas, key=probabilidade)
    return palavra_correta

# A função novo_corretor(). Vai adicionar a função gerador_turbinado()
def novo_corretor(palavra):
    palavras_geradas = gerador_palavras(palavra)
    palavras_turbinado = gerador_turbinado(palavras_geradas)
    todas_palavras = set(palavras_geradas + palavras_turbinado)
    candidatos = [palavra]
    for palavra in todas_palavras:
        if palavra in vocabulario:
            candidatos.append(palavra)
    palavra_correta = max(candidatos, key=probabilidade)
    return palavra_correta

# Função cria_dados_teste().cria uma lista de palavras para o teste.
def cria_dados_teste(nome_arquivo):
    lista_palavras_teste = []
    f = open(nome_arquivo, 'r', encoding = "utf8")
    for linha in f:
        correta, errada = linha.split()
        lista_palavras_teste.append((correta, errada))
    f.close()
    return lista_palavras_teste

# Função avaliador(). Vai avaliar o quanto nosso corretor é acertivo.
def avaliador(testes, vocabulario):
    numero_palavras = len(testes)
    acertou = 0
    desconhecida = 0
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        desconhecida += (correta not in vocabulario)
        if palavra_corrigida == correta:
            acertou += 1  
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    taxa_desconhecidas = round(desconhecida*100/numero_palavras, 2)
    print('Taxa de acerto {}% , desconhecidas {} % de {} palavras.'.format(taxa_acerto, taxa_desconhecidas, numero_palavras))

    
def avaliador_novo(testes, vocabulario):
    numero_palavras = len(testes)
    acertou = 0
    desconhecida = 0
    for correta, errada in testes:
        palavra_corrigida = novo_corretor(errada)
        desconhecida += (correta not in vocabulario)
        if palavra_corrigida == correta:
            acertou += 1  
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    taxa_desconhecidas = round(desconhecida*100/numero_palavras, 2)
    print('Taxa de acerto {}% , desconhecidas {} % de {} palavras.'.format(taxa_acerto, taxa_desconhecidas, numero_palavras))


total_palavras = len(lista_normalizada)
# Esse método .FreqDist() calcula a distribuição de frequencia das nossas palavras
frequencia = nltk.FreqDist(lista_normalizada)
vocabulario = set(lista_normalizada)
lista_teste = cria_dados_teste('palavras.txt')

In [56]:
palavra = 'njessa'
print(novo_corretor(palavra))
print(corretor(palavra))

essa
nessa


In [54]:
print(avaliador(lista_teste, vocabulario))
print(avaliador_novo(lista_teste, vocabulario))

Taxa de acerto 76.34% , desconhecidas 6.99 % de 186 palavras.
None
Taxa de acerto 55.38% , desconhecidas 6.99 % de 186 palavras.
None


**Podemos observar que utilizando a função novo_corretor() que utiliza o gerador_turbinado() temos uma taxa de acerto muito baixa porque ele devolve muitas palavras que tem mais de uma forma de ser correta como por exemplo (njessa) que o novo_corretor() nos trouxe "essa" e o corretor() nos trouxe "nessa".**

In [58]:
palavra = 'etsher'
print(novo_corretor(palavra))
print(corretor(palavra))

fisher
aetsher


In [59]:
palavra = 'oal'
print(novo_corretor(palavra))
print(corretor(palavra))

o
tal
